In [1]:
import pandas as pd

For now, I only want to use public hospitals.

In [2]:
# import hospitals to dataframes
public = pd.read_csv('4_LegalEntitySummaryPublicHospital.csv')
#private = pd.read_csv('4_LegalEntitySummaryNGOHospital.csv')

In [3]:
# inspect both datasets
public.head()

,Premises Name,Certification Service Type,Service Types,Total Beds,Premises Website,Premises Address Other,Premises Address,Premises Address Suburb/Road,Premises Address Town/City,Premises Address Post Code,DHB Name,Certificate Name,Certification Period (Months),Certificate/License End Date,Current Auditor,Legal Name,Legal Entity Address Other,Legal Entity Address,Legal Entity Address Suburb/Road,Legal Entity Address Town/City,Legal Entity Address Post Code,Legal Entity Postal Address,Legal Entity Postal Address Suburb/Road,Legal Entity Postal Address Town/City,Legal Entity Postal Address Post Code,Legal Entity Website,Unnamed: 26
0,Auckland City Hospital,Public Hospital,"Childrens health, Maternity, Surgical, Medical",1165,NaN,NaN,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
1,Auckland DHB X 3 Units - Mental Health,Public Hospital,Mental health,96,NaN,NaN,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
2,Buchanan Rehabilitation Centre,Public Hospital,Mental health,40,NaN,NaN,27 Sutherland Road,Point Chevalier,Auckland,1025,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
3,Greenlane Clinical Centre,Public Hospital,"Surgical, Medical",31,NaN,NaN,214 Green Lane West,Epsom,Auckland,1051,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
4,Tauranga Hospital,Public Hospital,"Psychogeriatric, Geriatric, Mental health, Chi...",360,NaN,NaN,829 Cameron Road,Tauranga South,Tauranga,3112,Bay of Plenty District Health Board,Bay of Plenty District Health Board,36,3-10-2023,The DAA Group Limited,Bay of Plenty District Health Board,NaN,829 Cameron Road,NaN,Tauranga,3112,Private Bag 12024,NaN,Tauranga,3143,NaN,NaN


In [ ]:
#private.head()

In [4]:
print(len(public))
#print(len(private))

83


There are 83 public hospitals and 72 private. Both dataset seem to have the same columns, hence I should be able to concatenate them into a single list.

In [ ]:
#hospitals = pd.concat([public, private])
#print(len(hospitals))

As expected, the concatenated list contains 155 records.

Instead of the above, I will only use public hospitals.

In [65]:
hospitals = public.copy()

In [66]:
hospitals.head(1)

,Premises Name,Certification Service Type,Service Types,Total Beds,Premises Website,Premises Address Other,Premises Address,Premises Address Suburb/Road,Premises Address Town/City,Premises Address Post Code,DHB Name,Certificate Name,Certification Period (Months),Certificate/License End Date,Current Auditor,Legal Name,Legal Entity Address Other,Legal Entity Address,Legal Entity Address Suburb/Road,Legal Entity Address Town/City,Legal Entity Address Post Code,Legal Entity Postal Address,Legal Entity Postal Address Suburb/Road,Legal Entity Postal Address Town/City,Legal Entity Postal Address Post Code,Legal Entity Website,Unnamed: 26
0,Auckland City Hospital,Public Hospital,"Childrens health, Maternity, Surgical, Medical",1165,NaN,NaN,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN


In [67]:
# Next, I will only keep the relevant columns
hospitals = hospitals[['Premises Name', 'Certification Service Type', ' Premises Address',
                       ' Premises Address Suburb/Road', ' Premises Address Town/City', 
                       ' Premises Address Post Code', ' DHB Name']]
# combine premises address with suburb/road
# but only if suburb/road is not empty
# iterate over the dataset
for index, row in hospitals.iterrows():
  if row[' Premises Address Suburb/Road'] != '':
    hospitals.at[index,' Premises Address'] = str(row[' Premises Address']) + ', ' + str(row[' Premises Address Suburb/Road'])
# and change Subur/Road to City
hospitals[' Premises Address Suburb/Road'] = hospitals[' Premises Address Town/City']
hospitals[' Premises Address Town/City'] = ''
# this is so that the name of the state matches with the rest of the data
# and rename them to match the format
hospitals = hospitals.rename(columns={'Premises Name': 'name', ' Premises Address': 'address',
                                      ' Premises Address Suburb/Road': 'city',
                                      ' Premises Address Town/City': 'state',
                                      ' Premises Address Post Code': 'zip',
                                      'Certification Service Type': 'type',
                                      ' DHB Name': 'county'})
# and add the missing columns
hospitals[''] = hospitals.index
hospitals['id'] = hospitals.index
hospitals['phone'] = ''
hospitals['ownership'] = ''
hospitals['emergency'] = ''
hospitals['quality'] = ''
# and reaarange them
hospitals = hospitals[['id', 'name', 'address', 'city', 'state', 'zip', 'county',
                       'phone', 'type', 'ownership', 'emergency', 'quality']]
# check that I have what I expected
hospitals.head()

,id,name,address,city,state,zip,county,phone,type,ownership,emergency,quality
0,0,Auckland City Hospital,"2 Park Road, Grafton",Auckland,,1023,Auckland District Health Board,,Public Hospital,,,
1,1,Auckland DHB X 3 Units - Mental Health,"2 Park Road, Grafton",Auckland,,1023,Auckland District Health Board,,Public Hospital,,,
2,2,Buchanan Rehabilitation Centre,"27 Sutherland Road, Point Chevalier",Auckland,,1025,Auckland District Health Board,,Public Hospital,,,
3,3,Greenlane Clinical Centre,"214 Green Lane West, Epsom",Auckland,,1051,Auckland District Health Board,,Public Hospital,,,
4,4,Tauranga Hospital,"829 Cameron Road, Tauranga South",Tauranga,,3112,Bay of Plenty District Health Board,,Public Hospital,,,


In [68]:
# I noticed that the leading zeros in zipcodes have been removed 
hospitals['zip'].unique()

array([1023, 1025, 1051, 3112, 3120, 7700, 8083, 8013, 8011, 7510, 7632,
       8024, 7300, 7608, 7430, 7400, 8022, 7420, 5032, 5022, 6021, 2025,
       2013, 2123, 2104, 2110, 2120, 4200, 4120, 4108, 5010, 3010, 3330,
       5510, 4414, 7020, 7010, 7007, 7120, 7201,  210,  310,  410,  110,
       7910, 9016, 9300, 9812, 9010, 4010, 4610, 4310, 3204, 3800, 3300,
       3920, 3910, 3500, 3420, 3240, 5910,  622,  610, 7825, 7805, 4501])

In [69]:
# I will first convert that row to string
hospitals['zip'] = hospitals['zip'].astype(str)
# and if the length is 3, add a leading zero
# iterate over the dataset
for index, row in hospitals.iterrows():
  if len(row['zip']) == 3:
    hospitals.at[index,'zip'] = '0' + row['zip']

Some states can be matched by looking at city name, e.g. 'Auckland'.
I can iterate through the dataframe and if the city name equals region name, use that as state name.

In [70]:
# set up a list of states
states = ['Northland', 'Auckland', 'Waikato', 'Manawatu-Wanganui',
       'Bay of Plenty', "Hawke's Bay", 'Gisborne', 'Taranaki',
       'Wellington', 'Area Outside', 'Tasman', 'Nelson', 'Marlborough',
       'Canterbury', 'West Coast', 'Otago', 'Southland']

In [71]:
# iterate over the dataset
for index, row in hospitals.iterrows():
  # I can match based on city name
  city = row['city']
  if city in states:
    hospitals.at[index,'state'] = state

In [72]:
# check how many states were matched
hospitals.loc[hospitals['state'] != ''].count()

id           23
name         23
address      23
city         23
state        23
zip          23
county       23
phone        23
type         23
ownership    23
emergency    23
quality      23
dtype: int64

23 states were matched matched.

Finally, I need to obtain the coordinates for each hospital. I can use GeoPy for this.

In [73]:
# install needed libraries
!pip install geopy 
!pip install Nominatim

In [74]:
# initiate geolocator
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent='myGeocoder')

In [75]:
# set up empty columns for latitude and longitude
hospitals['LAT'] = ''
hospitals['LON'] = ''

In [77]:
# iterate over all hospitals
for index, row in hospitals.iterrows():
    # get the location
    # I found that just looking by postcode produced the best result
    address = ' '.join([str(row['zip']), "New Zealand"])
    location = locator.geocode({address})
    # if location has been found
    if location != None:
      # add the coordinates to the dataframe
      hospitals.at[index,'LAT'] = location.latitude
      hospitals.at[index,'LON'] = location.longitude

In [78]:
# check if all locations were found
hospitals.loc[hospitals['LAT'] == ''].count()

id           0
name         0
address      0
city         0
state        0
zip          0
county       0
phone        0
type         0
ownership    0
emergency    0
quality      0
LAT          0
LON          0
dtype: int64

The coordinates for all hospitals were found.

The District Health Boards in New Zealand, don't correspond very well to the regions. As there are only 83 hospitals in this dataset, I will amend the remaining states in Excel.<br><br>
The 'Area Outside' does not have a hospital as it covers anything not covered elsewhere. I could either remove all areas that do not belong to a region or duplicate the records for one of the biggest hospitals in New Zealand and assign it to 'Area Outside', which is what I have done in Excel.

In [81]:
# I noticed that my index contains duplicates, I need to reset it.
hospitals = hospitals.reset_index()
hospitals['_'] = hospitals.index
hospitals['id'] = hospitals.index
hospitals = hospitals[['_', 'id', 'name', 'address', 'city', 'state', 'zip', 'county', 'phone',
       'type', 'ownership', 'emergency', 'quality', 'LAT', 'LON']]

## Save as csv


In [82]:
hospitals.to_csv('hospitals.csv', index=False)